In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer # for imputing missing values
from sklearn.preprocessing import StandardScaler # for standardizing data
from csvToExcel import toExcel
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, LSTM # for creating regular densely-connected NN layers and RNN layers

In [2]:
# Function to check for missing data
def checkMissingData():
    df = pd.read_csv('DataFolder/data.csv')
    print(df.info())
    
checkMissingData()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38472 entries, 0 to 38471
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Entity                           38472 non-null  object 
 1   Continent                        38472 non-null  object 
 2   Latitude                         38472 non-null  float64
 3   Longitude                        38472 non-null  float64
 4   Average temperature per year     38472 non-null  int64  
 5   Hospital beds per 1000 people    38472 non-null  float64
 6   Medical doctors per 1000 people  38472 non-null  float64
 7   GDP/Capita                       38472 non-null  float64
 8   Population                       38472 non-null  int64  
 9   Median age                       38472 non-null  int64  
 10  Population aged 65 and over (%)  38472 non-null  int64  
 11  Date                             38472 non-null  object 
 12  Daily tests       

In [3]:
# Fill missing values with mean
def missingDataHandler():
    df = pd.read_csv('DataFolder/data.csv')
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    # From the missing data analysis, we know that the columns 'Daily tests', 'Cases', 'Deaths' have missing values
    missingColumns = ['Daily tests', 'Cases', 'Deaths']
    for column in missingColumns:
        df[column] = imputer.fit_transform(df[column].values.reshape(-1, 1)) # fit_transform() returns a numpy array, so we need to convert it back to a pandas dataframe
    print(df.info())
    return df
    
dataFrame = missingDataHandler()
# toExcel(dataFrame, 'preprocessedData')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38472 entries, 0 to 38471
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Entity                           38472 non-null  object 
 1   Continent                        38472 non-null  object 
 2   Latitude                         38472 non-null  float64
 3   Longitude                        38472 non-null  float64
 4   Average temperature per year     38472 non-null  int64  
 5   Hospital beds per 1000 people    38472 non-null  float64
 6   Medical doctors per 1000 people  38472 non-null  float64
 7   GDP/Capita                       38472 non-null  float64
 8   Population                       38472 non-null  int64  
 9   Median age                       38472 non-null  int64  
 10  Population aged 65 and over (%)  38472 non-null  int64  
 11  Date                             38472 non-null  object 
 12  Daily tests       

In [4]:
# Get the data for Greece until 2020-12-31
def getGreeceData(df):
    df = df[df['Entity'] == 'Greece']
    df = df[df['Date'] <= '2020-12-31']
    return df

dataFrame2 = getGreeceData(dataFrame)

In [5]:
# Add new columns for positivity percent and daily cases
def addPositivityPercent(df):
    df['Daily Cases'] = abs(df['Cases'].diff())
    df['Positivity percent'] = df['Daily Cases'] / df['Daily tests']
    df['Date'] = pd.to_datetime(df['Date'])
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    df['Positivity percent'] = imputer.fit_transform(df['Positivity percent'].values.reshape(-1, 1))
    df['Daily Cases'] = imputer.fit_transform(df['Daily Cases'].values.reshape(-1, 1))
    return df

dataset = addPositivityPercent(dataFrame2)
# toExcel(dataset, 'GreeceData')
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 11802 to 12111
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Entity                           310 non-null    object        
 1   Continent                        310 non-null    object        
 2   Latitude                         310 non-null    float64       
 3   Longitude                        310 non-null    float64       
 4   Average temperature per year     310 non-null    int64         
 5   Hospital beds per 1000 people    310 non-null    float64       
 6   Medical doctors per 1000 people  310 non-null    float64       
 7   GDP/Capita                       310 non-null    float64       
 8   Population                       310 non-null    int64         
 9   Median age                       310 non-null    int64         
 10  Population aged 65 and over (%)  310 non-null    int64  

In [6]:
# Function to get all the data for Greece
def allGreeceData(df):
    df = df[df['Entity'] == 'Greece']
    df['Daily Cases'] = abs(df['Cases'].diff())
    df['Positivity percent'] = df['Daily Cases'] / df['Daily tests']
    df['Date'] = pd.to_datetime(df['Date'])
    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    df['Positivity percent'] = imputer.fit_transform(df['Positivity percent'].values.reshape(-1, 1))
    df['Daily Cases'] = imputer.fit_transform(df['Daily Cases'].values.reshape(-1, 1))
    return df

dataFrame3 = allGreeceData(dataFrame)
print(dataFrame3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 369 entries, 11802 to 12170
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Entity                           369 non-null    object        
 1   Continent                        369 non-null    object        
 2   Latitude                         369 non-null    float64       
 3   Longitude                        369 non-null    float64       
 4   Average temperature per year     369 non-null    int64         
 5   Hospital beds per 1000 people    369 non-null    float64       
 6   Medical doctors per 1000 people  369 non-null    float64       
 7   GDP/Capita                       369 non-null    float64       
 8   Population                       369 non-null    int64         
 9   Median age                       369 non-null    int64         
 10  Population aged 65 and over (%)  369 non-null    int64  

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [7]:
# Here starts the training of the model
posPer = dataset['Positivity percent'].values
# print(posPer.shape)

In [8]:
# Standardize the data
sc_data = StandardScaler()
posPer = sc_data.fit_transform(posPer.reshape(-1, 1))

In [9]:
# Split the data into training and testing sets
sequence = posPer.shape[0] - 1
X = []
y = []
for i in range(sequence, len(posPer)):
    X.append(posPer[i-sequence:i])
    y.append(posPer[i])
X = np.array(X)
y = np.array(y)
# print(X.shape, y.shape)

In [10]:
# Prepare the model
model = Sequential()
model.add(LSTM(32, input_shape=(sequence, 1))) # LSTM layer with 32 neurons (LSTM = Long Short Term Memory) and input shape (number of time steps, number of features)
model.add(Dense(1)) # Dense layer with 1 neuron (output layer)
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error') # Adam optimizer and mean squared error loss function
# Fit keras model on the dataset
model.fit(X, y,  epochs=200, batch_size=1000) # 200 epochs is the number of times the model will cycle through the data and 1000 is the number of samples per gradient update

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 7.4284e-04
Epoch 2/200
1/1 [==============================] - 0s 85ms/step - loss: 1.3643e-04
Epoch 3/200
1/1 [==============================] - 0s 116ms/step - loss: 5.7590e-06
Epoch 4/200
1/1 [==============================] - 0s 74ms/step - loss: 1.5320e-04
Epoch 5/200
1/1 [==============================] - 0s 85ms/step - loss: 2.6254e-04
Epoch 6/200
1/1 [==============================] - 0s 112ms/step - loss: 2.3202e-04
Epoch 7/200
1/1 [==============================] - 0s 98ms/step - loss: 1.3107e-04
Epoch 8/200
1/1 [==============================] - 0s 81ms/step - loss: 3.9701e-05
Epoch 9/200
1/1 [==============================] - 0s 112ms/step - loss: 7.4851e-07
Epoch 10/200
1/1 [==============================] - 0s 116ms/step - loss: 1.5910e-05
Epoch 11/200
1/1 [==============================] - 0s 83ms/step - loss: 5.7707e-05
Epoch 12/200
1/1 [==============================] - 0s 87ms/step - loss: 9.1966e-05

In [11]:
# Predict the positivity percent for the next 3 days
def predictNextDays(date):
    numOfDays = pd.to_datetime(date) - dataFrame3.iloc[posPer.shape[0] - 1]['Date'] # number of days from the last date in the dataset
    last_sequence = posPer[-sequence:]
    predictions = []
    for _ in range(numOfDays.days):
        input_sequence = last_sequence.reshape(1, sequence, 1) # reshape the input sequence to fit the model
        prediction = model.predict(input_sequence) # predict the positivity percent for the next day
        predictions.append(prediction[0]) # append the prediction to the predictions list
        last_sequence = np.concatenate((last_sequence[1:], prediction), axis=0) # concatenate is used to add the prediction to the last_sequence array 
    predictions = sc_data.inverse_transform(predictions) # inverse transform the predictions to get the actual values
    return predictions

predictions = predictNextDays('2021-01-03') # predict the positivity percent for the next 3 days (until 2021-01-03)

1/1 [==============================] - 0s 37ms/step


In [12]:
print("Predict positivity percent for the next 3 days:\n")
for i, prediction in enumerate(predictions):
    print(f"Date: {dataFrame3.iloc[posPer.shape[0] + i]['Date'].date()}")
    print(f"Prediction {i+1}: {round(prediction[0] * 100, 2)} %")
    print(f"Actual value: {round(dataFrame3.iloc[posPer.shape[0] + i]['Positivity percent'] * 100, 2)} %\n")

Predict positivity percent for the next 3 days:

Date: 2021-01-01
Prediction 1: 2.91 %
Actual value: 2.08 %

Date: 2021-01-02
Prediction 2: 2.9 %
Actual value: 5.46 %

Date: 2021-01-03
Prediction 3: 2.89 %
Actual value: 4.59 %

